# Gymnasium 

Gymnasium, es una librería de aprendizaje reforzado, por lo que utiliza el bucle agente-entorno:


<img src="./media/AE_loop.png" width="350px"/>

Los elementos que aparecen en la imagen, dentro de gymnasium funcionarán de la siguiente forma:

## Enviroment
TODO MIRAR LO DE MULTIAGENTE DE PETTINGZOO <br> <br>
En esta clase, se encapsula el entorno que puede ser observado por el agente, se va a trabajar con el entorno *LunarLander*, para demostrar todas las propiedades de las clases entorno:

<img src="./media/LunarLander.gif" width="350px"/>


In [3]:
import gymnasium as gym
#Carga del entorno para trabajar con la clase
lunarLander = gym.make('LunarLander-v2', render_mode="human") 

/home/alberto/anaconda3/envs/gymnasium/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Una clase del tipo entorno, consta de los siguientes atributos para facilitar su implementación:

### Atributos

- **action_space:** es el objeto de espacios correspondiente a las acciones válidas.

In [10]:
print(lunarLander.action_space)
print(type(lunarLander.action_space))

Discrete(4)
<class 'gymnasium.spaces.discrete.Discrete'>


Como se puede apreciar *action_space* es un objeto del tipo Discrete, esta clase también es de gymnasium y representa un espacio finito de muchos elementos, concretamente números enteros. <br>
Estos números, posteriormente representarán acciones (a definir por el modelo). Por ejemplo, las acciones del modelo anterior serían:
- **0:** no hacer nada
- **1:** encender el motor izquierdo
- **2:** encender el motor principal (ambos)
- **3:** encender el motor derecho

Para crear un objeto de este tipo, será necesario indicar el número de elementos del espacio (4 en el caso anterior), el elemento más pequeño del espacio (inicio) y opcionalmente, una semilla en caso de querer que sea aleatorio.

In [21]:
d = gym.spaces.Discrete(2,start=0)
print(d)

Discrete(2)


- **observation_space:** es el espacio de observaciones de un entorno.

In [66]:
#print("{}".format(lunarLander.observation_space))
print(lunarLander.observation_space.high)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


El espacio de observaciones, es un objeto tipo *Box*, el cual representa el producto cartesiano de dos intervalos n:
<img src="./media/productoCartesiano.png" width="350px"/>
Los límites de los intervalos pueden ser de una de las siguientes formas, $[a,b], (-\infty,b],[a,\infty) o (-\infty,\infty) $
- 


In [61]:
import numpy as np
b = gym.spaces.Box(low=-2.0, high=np.array([2.0, 4.0]), dtype=np.float32)
print(b)

Box(-2.0, [2. 4.], (2,), float32)



- **reward_range:** es una tupla que hace referencia a la máxima y mínima recompensa posible para el agente en un momento (por defecto es en ($-\infty,+\infty)$


In [82]:
print(lunarLander.reward_range) 

#No se ha encontrado ningún modelo que tenga parámetros distintos a 
#los de por defecto



(-inf, inf)


- **spec:** Contiene la información que ha utilizado el método make para inicializar el entorno.


In [83]:
print(lunarLander.spec) 

EnvSpec(id='LunarLander-v2', entry_point='gymnasium.envs.box2d.lunar_lander:LunarLander', reward_threshold=200, nondeterministic=False, max_episode_steps=1000, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='LunarLander', version=2)


- **metadata:** Son los metadatos del entorno
- **np_random:** El generador de números aleatorios para el entorno

In [86]:
print(str(lunarLander.metadata) + "\n" ) 
print(lunarLander.np_random)

{'render_modes': ['human', 'rgb_array'], 'render_fps': 50}

Generator(PCG64)


### Métodos
Los métodos principales con los que se puede trabajar con el entorno son:


- **reset(self, seed) &rarr; tuple[ObsType, dict[str, Any]]:** El entorno vuelve al estado inicial y devuelve la primera observación del agente e información como métricas, debug...

En el caso de este entorno, es un vector de 8 dimensiones y cada una representa:
1. Coordenada x de la nave
2. Coordenada y de la nave
3. Velocidad linear en x de la nave
4. Velocidad linear en y de la nave
5. Ángulo de la nave
6. Velocidad angular de la nave
7. Motor izquierdo en contacto con el suelo (booleano)
8. Motor derecho en contacto con el suelo (booleano)


In [42]:
observation, info = lunarLander.reset()

print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Motor derecho en contacto?: {}".format(observation[7]))
print("Informacion extra: {}".format(info))


La coordenada x de la nave es: -0.006809711456298828
La coordenada y de la nave: 1.402424693107605
La velocidad linear en x de la nave es: -0.6897678375244141
La velocidad linear en y de la nave es: -0.37759706377983093
El ángulo de la nave es: 0.007897558622062206
La velocidad angular de la nave es: 0.1562427431344986
Motor izquierdo en contacto?: 0.0
Motor derecho en contacto?: 0.0
Informacion extra: {}


- **step(self, action: ActType) &rarr; tuple[ObsType, SupportsFloat, bool, bool, dict[str, Any]]:** Actualiza el entorno con la acción pasada por parámetros, devolviendo la siguiente observación del agente, la recompensa obtenida, si la ejecución del entorno ha terminado debido a la última acción e información del entorno sobre la acción e información de depuración.

In [45]:
observation, reward, terminated, truncated, info = lunarLander.step(0)
#No se hace nada 
print("No se hace nada (en este caso unicamente se cae la nave): \n\n")
print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Motor derecho en contacto?: {}".format(observation[7]))


observation, reward, terminated, truncated, info = lunarLander.step(2) 
#Se encienden los dos motores
print("\n\nSe encienden los dos motores: \n\n")
print("La coordenada x de la nave es: {}".format(observation[0]))
print("La coordenada y de la nave: {}".format(observation[1]))
print("La velocidad linear en x de la nave es: {}".format(observation[2]))
print("La velocidad linear en y de la nave es: {}".format(observation[3]))
print("El ángulo de la nave es: {}".format(observation[4]))
print("La velocidad angular de la nave es: {}".format(observation[5]))
print("Motor izquierdo en contacto?: {}".format(observation[6]))
print("Motor derecho en contacto?: {}".format(observation[7]))

No se hace nada (en este caso unicamente se cae la nave): 


La coordenada x de la nave es: -0.04036521911621094
La coordenada y de la nave: 1.3527436256408691
La velocidad linear en x de la nave es: -0.6709190011024475
La velocidad linear en y de la nave es: -0.47274255752563477
El ángulo de la nave es: 0.03860168531537056
La velocidad angular de la nave es: 0.11921069771051407
Motor izquierdo en contacto?: 0.0
Motor derecho en contacto?: 0.0


Se encienden los dos motores: 


La coordenada x de la nave es: -0.046959780156612396
La coordenada y de la nave: 1.343008041381836
La velocidad linear en x de la nave es: -0.6659213304519653
La velocidad linear en y de la nave es: -0.43286916613578796
El ángulo de la nave es: 0.04500451311469078
La velocidad angular de la nave es: 0.12806841731071472
Motor izquierdo en contacto?: 0.0
Motor derecho en contacto?: 0.0


En este caso, se puede apreciar que la coordenada *y* desciende la primera vez 0.5 al no realizar ninguna acción. <br> <br>
Después de realizar el segundo paso (encender ambos motores), el descenso en el eje *y* es de 0.1 (mucho menor que en la primera ejecución), por lo que se demuestra que se ha ejecutado la acción, ya que al encenderse ambos motores no se ha producido un descenso tan grande como en la primera.

- **render(self) &rarr; RenderFrame | list[RenderFrame] | None:** Renderiza el entorno para que se pueda visualizar. Es necesario que se especifique el tipo de renderizado a la hora de realizar el *make* 

In [4]:
lunarLander.reset()
lunarLander.render()

- **close(self):** cierra el entorno y acaba la ejecución (similar a cuandos se cierra un fichero).

In [5]:
lunarLander.close()